# Morenney Rock Glacier Radar Imagery Example

## Prepare Data for Bulk Download

Akhdefo processes radar imagery that has been corrected for radiometric and terrain distortions. Its function, 'download_RTC,' enables the downloading of Synthetic Aperture Radar (SAR) products from ASF's HyP3 platform. Users can easily order RTC, AutoRIFT, and InSAR products with just a few lines of Python script.
Firstly, users are advised to visit [ASF's website](https://search.asf.alaska.edu/#/?maxResults=250) to download metadata for their chosen time-series product granules. To assist users, example screenshots are provided below as a guide to navigate through the process.


![Step 1](data/asf1.png)


![Step 1](data/asf2.png)

## Import AkhDefo Packages.

In [1]:

#Functions for data preparations
import akhdefo_functions
from akhdefo_functions import download_RTC
from akhdefo_functions import read_data_prep
from akhdefo_functions import move_files_with_string
from akhdefo_functions import Crop_to_AOI

#Functions data processing
from akhdefo_functions import scatter_area_mask
from akhdefo_functions import Raster_Correction
from akhdefo_functions import Optical_flow_akhdefo

#Functions for shapefile point data interpolation to raster geotif
#Function Autovariogram use Kriging method it fits the data to different variogram models and automatically use the best fit variogram model (Computationally Heavy but you can process data in chunks)
from akhdefo_functions import Auto_Variogram

# interpoate_xyz uses the following interpolation methods (nearest, linear and cubic)
from akhdefo_functions import interpolate_xyz

#Functions for data Visualization and Analysis
from akhdefo_functions import MeanProducts_plot_ts
from akhdefo_functions import plot_stackNetwork
from akhdefo_functions import akhdefo_viewer
from akhdefo_functions import akhdefo_dashApp

## Download Data using EarthData ASF Account

In [ ]:

''' 

First Run the below command and make sure {download=False} to see the granule paths and frames. 

Its highly recommended to use data frame same path for the entire time-series for more accurate results
 
'''


download_RTC( prompt=True, asf_datapool_results_file= './data/morenny/radar/asf-datapool-results-2023-12-19_04-01-18.csv', 
            save_dir= './data/morenny/radar/path129', job_name= './data/morenny/radar/RockGlacier_asc', 
            dem_matching= False, 
            include_dem= False, 
            include_inc_map= True, 
            include_rgb = False, 
            include_scattering_area= True, 
            scale = 'power', 
            resolution = 20, 
            speckle_filter= False,
            radiometry='gamma0',
            dem_name='copernicus', limit=None , download=False , path_number=129, RTC=False)

## Extract downloaded zip files

In [ ]:
#This functions extracts data from all the granuples zip directory.
read_data_prep(zip_dir='./data/morenny/radar/path129_rtc/',
                                 image_dir='./data/morenny/radar/RS_dir_path129', 
                                 ext_image_file='VV.tif', udm_mask_dir='./data/morenny/radar/scatter_dir', ext_udm_mask_file='area.tif')

In [ ]:
# This function will move all the data from each granules subdir to a single folder
move_files_with_string(source_dir='./data/morenny/radar/RS_dir_path129/', dest_dir='./data/morenny/radar/RS_path129', search_string='.tif')
move_files_with_string(source_dir='./data/morenny/radar/scatter_dir/', dest_dir='./data/morenny/radar/scatter_rs', search_string='.tif')


In [ ]:
'''  
Optional:

Create an accumulated scatter area mask from a set of raster images based on a given threshold. 
the input dataset is taken from ASF RTC processing.
The scattering area for each pixel in the RTC image in square meters. 
The values are calculated based on the effectively illuminated gamma-0 terrain surface using a digital elevation model,
the local incidence angle map, and the layover-shadow map. see detailes 
at the following website https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#scattering-area-map
The function processes each raster image in the input folder, crops it based on the provided AOI from the shapefile,
normalizes the cropped raster, and then converts the normalized image to a binary mask based on the scatter_percentageArea_threshold. 
The binary masks from each raster are then accumulated to generate the final scatter area mask.

'''

input_folder = "./data/morenny/radar/scatter_rs"
output_folder = "./data/morenny/radar/scatter_masks_20"
plot_folder = "./data/morenny/radar/scatter_mask_plots_20"
shapefile_path = "./data/morenny/AOI.shp"
scatter_Area_threshold=0
vegetation_mask_path=None


scatter_area_mask(input_folder, output_folder, plot_folder, shapefile_path, scatter_Area_threshold, vegetation_mask_path)



In [5]:

''' 
Crop all raster to Area of interest bounding box


'''
Crop_to_AOI(Path_to_WorkingDir='./data/morenny/radar/RS_path129/', 
                              Path_to_AOI_shapefile='./data/morenny/AOI.shp', 
                              output_CroppedDir='./data/morenny/radar/RS_cropped_path129', file_ex='.tif')



0it [00:00, ?it/s]


In [2]:
import warnings
warnings.filterwarnings("ignore")


Raster_Correction(input_path="./data/morenny/radar/RS_cropped_path129", output_path="./data/morenny/radar/RS_cropped_path129_filt", limit=None,
                                    lowpass_kernel_size=None, bilateral_win_size=7, bilateral_sigma_color=75,
                                    bilateral_sigma_spatial=75,
                                    clip_percentiles=[2, 98], optical=False, 
                                    scale='power', Vegetation_mask=None)

Processing images: 100%|██████████| 78/78 [01:10<00:00,  1.11image/s]


In [4]:
Optical_flow_akhdefo(input_dir='./data/morenny/radar/RS_cropped_path129_filt', output_dir='./data/morenny/radar/geo_path129', AOI='./data/morenny/AOI.shp', zscore_threshold=2, 
                                       ssim_thresh=0.65, image_resolution='20m', interpolate='linear', 
                                       show_figure=False, point_size=2, dem_path='./data/morenny/dem.tif', smoothing_kernel_size=None, 
                                       Vegetation_mask=None, VEL_scale='year', VEL_Mode='linear', good_match_option=0.75, 
                                       hillshade_option=True, shapefile_output=True, max_triplet_interval=30, pixel_size=100, num_chunks=10, overlap_percentage=0, pyr_scale=0.5, levels=15, 
                                       winsize=128, iterations=7, poly_n=7, poly_sigma=1.5,
                                       flags=1, master_reference='single', 
                                       selection_Mode='pair', start_date=None, end_date=None)

Processing:   1%|▏         | 1/77 [00:01<01:36,  1.26s/it]

Skipping computation for 20150717 to 20150903 as the time interval is larger than 30 days.
Skipping computation for 20150903 to 20160605 as the time interval is larger than 30 days.


Processing:   9%|▉         | 7/77 [00:05<01:03,  1.10it/s]

Skipping computation for 20160909 to 20170612 as the time interval is larger than 30 days.


Processing:  22%|██▏       | 17/77 [00:15<01:04,  1.08s/it]

...

ReferencePoint x, y: (662340.5187988281, 4762192.402801514), VEL, VEL_STD: (0.4157828553367713, 0.025150325271425335)
processing geo_path129/temp_shapefile_dir/20150623_20230921_20230921_2DVEL.shp completed...  

processing geo_path129/temp_shapefile_dir/20150623_20230921_20230921_N.shp started...  

571


Processing: Update Shapefiles :  67%|██████▋   | 2/3 [00:29<00:14, 14.53s/it]

ReferencePoint x, y: (662340.5187988281, 4762192.402801514), VEL, VEL_STD: (0.05780702361658835, 0.011082207690007652)
processing geo_path129/temp_shapefile_dir/20150623_20230921_20230921_N.shp completed...  

processing geo_path129/temp_shapefile_dir/20150623_20230921_20230921_E.shp started...  

571


Processing: Update Shapefiles : 100%|██████████| 3/3 [00:43<00:00, 14.52s/it]

ReferencePoint x, y: (662340.5187988281, 4762192.402801514), VEL, VEL_STD: (-0.08806014301128967, 0.006105641359759221)
processing geo_path129/temp_shapefile_dir/20150623_20230921_20230921_E.shp completed...  

Total Days: 3012
file used geo_path129/temp_shapefile_dir\updated_shapefiles\20150623_20230921_20230921_2DVEL.shp.xml


In [6]:
akhdefo_dashApp(
Path_to_Shapefile='./data/morenny/radar/geo_path129/temp_shapefile_dir/updated_shapefiles/20150623_20230921_20230921_2DVEL.shp',
                                  port=8057, BaseMap=True, basemap_type='image')